In [1]:
import pandas as pd
import recsys as rs
import json
import notipy
import TopSimilarRecommender as TSR

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
fit_dict = {'tracks_info' : tr_info,
            'attributes' : ['artist_id', 'album', 'tags'],
            'tgt_tracks' : tgt_tr,
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : tgt_pl,
                  'train_playlists_tracks_pairs' : train, 
                  'normalize' : False}

In [4]:
rec = TSR.TopSimilarRecommender()
rec.fit(**fit_dict)
notipy.notify('Model fitted!')
print('Model fitted!')

Fixed dataset
[    78    109    144 ..., 567363 568165 570588]
(17536,)


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
recommendetions = rec.recommend(**recommend_dict)
notipy.notify('Recommending completed!')
print('Recommending completed!')

In [ ]:
#map_eval = rs.evaluate(recommendetions, test, 'MAP')
#notipy.notify('Evaluation completed!')
#print('Evaluation completed!')

In [ ]:
run_data = {'recommender_type' : rec.__class__.__name__,
            'fit_parameters' : fit_dict,
            'recommend_parameters' : recommend_dict,
            'evaluation_result' : 0}

In [ ]:
with open('runs_data.json', 'w') as fp:
    json.dump(run_data, fp)
notipy.notify('Run data saved!')
print('Run data saved!')